Untuk menjalankan buku nota berikut, jika anda belum melakukannya, anda perlu menyebarkan model yang menggunakan `text-embedding-ada-002` sebagai model asas dan tetapkan nama penyebaran di dalam fail .env sebagai `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Seterusnya, kita akan memuatkan Indeks Embedding ke dalam Dataframe Pandas. Indeks Embedding disimpan dalam fail JSON yang dipanggil `embedding_index_3m.json`. Indeks Embedding mengandungi Embeddings untuk setiap transkrip YouTube sehingga akhir Okt 2023.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Seterusnya, kita akan mencipta fungsi yang dipanggil `get_videos` yang akan mencari Indeks Embedding untuk pertanyaan. Fungsi ini akan memulangkan 5 video teratas yang paling serupa dengan pertanyaan tersebut. Fungsi ini berfungsi seperti berikut:

1. Pertama, salinan Indeks Embedding dibuat.
2. Seterusnya, Embedding untuk pertanyaan dikira menggunakan OpenAI Embedding API.
3. Kemudian satu lajur baru dicipta dalam Indeks Embedding yang dipanggil `similarity`. Lajur `similarity` mengandungi kesamaan kosinus antara Embedding pertanyaan dan Embedding untuk setiap segmen video.
4. Seterusnya, Indeks Embedding ditapis mengikut lajur `similarity`. Indeks Embedding ditapis untuk hanya memasukkan video yang mempunyai kesamaan kosinus lebih besar atau sama dengan 0.75.
5. Akhir sekali, Indeks Embedding disusun mengikut lajur `similarity` dan 5 video teratas dipulangkan.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Fungsi ini sangat mudah, ia hanya mencetak hasil carian.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Pertama, Indeks Embedding dimuatkan ke dalam Dataframe Pandas.
2. Seterusnya, pengguna diminta untuk memasukkan pertanyaan.
3. Kemudian fungsi `get_videos` dipanggil untuk mencari Indeks Embedding bagi pertanyaan tersebut.
4. Akhir sekali, fungsi `display_results` dipanggil untuk memaparkan keputusan kepada pengguna.
5. Pengguna kemudian diminta untuk memasukkan pertanyaan lain. Proses ini berterusan sehingga pengguna memasukkan `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ms.png)

Anda akan diminta untuk memasukkan pertanyaan. Masukkan pertanyaan dan tekan enter. Aplikasi akan memulangkan senarai video yang berkaitan dengan pertanyaan tersebut. Aplikasi juga akan memulangkan pautan ke tempat dalam video di mana jawapan kepada soalan itu terletak.

Berikut adalah beberapa pertanyaan untuk dicuba:

- Apakah Azure Machine Learning?
- Bagaimana rangkaian neural konvolusional berfungsi?
- Apakah rangkaian neural?
- Bolehkah saya menggunakan Jupyter Notebooks dengan Azure Machine Learning?
- Apakah ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk ketepatan, sila ambil maklum bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang sahih. Untuk maklumat penting, terjemahan profesional oleh manusia adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
